# Imports

In [3]:
import fitz
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import numpy
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from pyresparser import ResumeParser

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Processing

In [5]:
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stopword = set(stopwords.words('english'))

In [6]:
# Load model
w2vmodel = Word2Vec.load("model.pkl")

In [7]:
# Find a word that exists in the model to be used a default
default_word = w2vmodel.wv.most_similar("computer")[0][0]

In [8]:
def cleaned_resume_list_variation(resume_path):
    with fitz.open(resume_path) as doc:
        resume = ""
        for page in doc:
            resume += page.getText()

    resume = resume.lower()    
    
    sent = resume.split('\n')
    
    full_list = []
    
    for s in sent:
        for word in word_tokenize(s):
            if word.isalpha() and word not in stopword:
                full_list.append(lemmatizer.lemmatize(word))      
        
    return full_list

In [9]:
def getwordVec(word, model):
    sample = model.wv[default_word]
    vec = [0]*len(sample)
    try:
        vec = model.wv[word]
    except:
        vec = [0]*len(sample)
        
    return vec

In [10]:
def getPhraseEmbedding(phrase,embeddingmodel):
    samp = getwordVec(default_word, embeddingmodel)
    vec=numpy.array((0)*len(samp));
    den=0; 
    for word in phrase:        
        den = den + 1; 
        vec = vec+numpy.array(getwordVec(word,embeddingmodel));        
    return vec.reshape(1, -1)

In [17]:
JD = cleaned_resume_list_variation("../Code/resume/JD.pdf")
JD

['job',
 'description',
 'key',
 'responsibility',
 'responsible',
 'scaling',
 'analytics',
 'capability',
 'across',
 'internal',
 'discipline',
 'guide',
 'strategic',
 'direction',
 'regard',
 'analytics',
 'organize',
 'analyze',
 'large',
 'diverse',
 'data',
 'set',
 'across',
 'multiple',
 'platform',
 'identify',
 'key',
 'insight',
 'leverage',
 'inform',
 'influence',
 'product',
 'strategy',
 'technical',
 'interaction',
 'vendor',
 'partner',
 'technical',
 'capacity',
 'approach',
 'deliverable',
 'develops',
 'proof',
 'concept',
 'prove',
 'disprove',
 'validity',
 'concept',
 'working',
 'part',
 'business',
 'identify',
 'analytical',
 'requirement',
 'formalize',
 'approach',
 'reliable',
 'relevant',
 'accurate',
 'efficient',
 'reporting',
 'requirement',
 'designing',
 'implementing',
 'advanced',
 'statistical',
 'testing',
 'customized',
 'problem',
 'solving',
 'deliver',
 'concise',
 'verbal',
 'written',
 'explanation',
 'analysis',
 'senior',
 'management',


In [12]:
wvJD = getPhraseEmbedding(JD, w2vmodel)

In [13]:
def similarityInterval(similarity):
    if similarity>80:
        return '100-80'
    elif similarity>60:
        return '80-60'
    elif similarity>40:
        return '60-40'
    elif similarity>20:
        return '40-20'
    else:
        return '20-0'

In [14]:
rankings = []
for i in range(1,21):
    resume_path = "../Code/resume/resume%d.pdf" % i
    resume = cleaned_resume_list_variation(resume_path)
    wvResume = getPhraseEmbedding(resume, w2vmodel)
    similarity = cosine_similarity(wvResume, wvJD)[0][0]*100
    name = ResumeParser(resume_path).get_extracted_data()['name']
    interval = similarityInterval(similarity)
    rankings.append({'ID': i, 'Name': name, 'Similarity': similarity, 'Interval': interval})

In [15]:
sorted_rankings = sorted(rankings, key=lambda k: k['Similarity'], reverse=True) 

In [16]:
sorted_rankings

[{'ID': 10,
  'Name': 'Brian Tong',
  'Similarity': 56.649609288844296,
  'Interval': '60-40'},
 {'ID': 8,
  'Name': 'WONG Man',
  'Similarity': 50.61806785724694,
  'Interval': '60-40'},
 {'ID': 7,
  'Name': 'Aman Kumar',
  'Similarity': 45.476051852329824,
  'Interval': '60-40'},
 {'ID': 11,
  'Name': 'Queen’s',
  'Similarity': 44.31801980637177,
  'Interval': '60-40'},
 {'ID': 2,
  'Name': 'RAJESH HEGDE',
  'Similarity': 43.297874666856515,
  'Interval': '60-40'},
 {'ID': 17,
  'Name': 'Kendrick Chia',
  'Similarity': 42.3402301839465,
  'Interval': '60-40'},
 {'ID': 4,
  'Name': 'Srinath Ramachandran',
  'Similarity': 42.01052868175263,
  'Interval': '60-40'},
 {'ID': 1,
  'Name': 'KALPAN MUKHERJEE',
  'Similarity': 36.76521089550706,
  'Interval': '40-20'},
 {'ID': 15,
  'Name': 'Curriculum Vitae',
  'Similarity': 36.537036137565316,
  'Interval': '40-20'},
 {'ID': 19,
  'Name': 'NATALIE NGAI',
  'Similarity': 33.55476772445412,
  'Interval': '40-20'},
 {'ID': 12,
  'Name': 'PIAMK